## Импорты

In [4]:
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26822 sha256=3827474085b14c924ae709601f468ae0d5e52f7be3335983151a05808107f289
  Stored in directory: c:\users\anastasia\appdata\local\pip\cache\wheels\5c\4b\ec\41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql
Processing c:\users\anastasia\appdata\local\pip\cache\wheels\5c\4b\ec\41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f\pandasql-0.7.3-py3-none-any.whl
  Attempting uninstall: pandasql
    Found existing installation: pandasql 0.7.3
    Uninstalling pandasql-0.7.3:
      Successfully uninstalled pandasql-0.7.3


In [6]:
import pandas as pd
from os.path import join
from glob import glob
import xgboost as xgb
from sklearn.metrics import f1_score
import pandasql as ps
from datetime import datetime, date, timedelta
from tqdm.notebook import tqdm as tqdm_notebook
import catboost

pd.set_option('display.max_columns', None)

In [7]:
DATADIR = '/Users/Anastasia/Desktop/python/Hackaton/DATA'


## Посмотрим на таблицы

### train

In [9]:
train_df = pd.read_csv(join(DATADIR, 'train', 'train.csv'))

In [8]:
train_df = pd.read_csv(join(DATADIR, 'train', 'train.csv'))
train_df['order_completed_at'] = train_df['order_completed_at'].str.extract('\d{4}-(\d{2})')[0].astype(int)
print(train_df.shape)
train_df.head()

(1159576, 3)


,phone_id,order_completed_at,target
0,19843,1,1.0
1,19843,2,0.0
2,19843,3,0.0
3,19843,4,0.0
4,19843,5,0.0


Тут заказы с января по июль 2020 года. Надо сделать предсказание для людей из файла sample_submission на сентябрь 2020.  

In [13]:
train_df['order_completed_at'].value_counts().sort_index()

2020-01     59652
2020-02     75985
2020-03    106791
2020-04    159845
2020-05    218569
2020-06    254285
2020-07    284449
Name: order_completed_at, dtype: int64

### Address

In [14]:
addr_df = pd.read_csv(join(DATADIR, 'misc', 'addresses.csv'))
addr_df.head()

,id,phone_id
0,41,1
1,8531,2
2,2982657,2
3,3271801,2
4,3648206,2


### Shipments

In [15]:
ship_df = pd.concat([pd.read_csv(file) for file in glob(join(DATADIR, 'shipments', '*'))], axis=0, ignore_index=False)

for col in ['order_created_at', 'order_completed_at', 'shipment_starts_at', 'shipped_at']:
    ship_df[col] = pd.to_datetime(ship_df[col], format='%Y-%m-%d %H:%M:%S')

print(ship_df.shape)
ship_df.head()

(2260686, 20)


,user_id,ship_address_id,shipment_id,order_id,order_created_at,order_completed_at,shipment_starts_at,retailer,s.order_state,shipment_state,s.city_name,s.store_id,total_cost,rate,dw_kind,promo_total,total_weight,platform,os,shipped_at
0,11019,171906,322307,2156687,2017-08-03 21:25:23,2020-02-18 14:07:00,2020-02-20 07:00:00,METRO,complete,shipped,Москва,21,168.0,0,courier,0.0,30170,web,windows,2020-02-20 08:08:54
1,62278,278832,387023,3021953,2018-03-02 17:22:04,2020-01-03 13:09:26,2020-01-03 17:00:00,METRO,complete,shipped,Москва,10,98.0,0,courier,-150.0,11305,web,windows,2020-01-03 18:10:40
2,905126,468407,388943,3018198,2018-02-28 11:32:47,2020-02-12 12:39:28,2020-02-13 11:00:00,METRO,complete,shipped,Москва,21,98.0,5,courier,0.0,13589,app,ios,2020-02-13 12:33:53
3,21412,61962,421048,3030227,2018-03-07 20:37:27,2020-01-25 11:58:56,2020-01-25 18:00:00,METRO,complete,shipped,Москва,8,158.0,0,courier,0.0,9726,web,mac,2020-01-25 19:55:32
4,42110,378297,442659,2923996,2017-12-24 11:19:04,2020-01-07 14:30:44,2020-01-07 19:00:00,METRO,complete,shipped,Москва,2,163.0,0,courier,0.0,30323,web,windows,2020-01-07 19:51:37


In [16]:
ship_df['order_created_at_month'] = ship_df['order_created_at'].dt.month
ship_df['order_completed_at_month'] = ship_df['order_completed_at'].dt.month

### user_profiles

In [17]:
users_df = pd.read_csv(join(DATADIR, 'misc', 'user_profiles.csv')).dropna(subset=['gender', 'bdate'], how='all')
users_df = users_df.drop_duplicates()
print(users_df.shape)

(48329, 3)


In [18]:
def get_age(bdate):
    if not isinstance(bdate, str):
        return None
    if '.' in bdate:
        sep = '.'
    elif '-' in bdate:
        sep = '-'
    else:
        sep = '/'
    vals = bdate.split(sep)
    if len(vals) < 3:
        return None
    vals = list(map(int, vals))
    if len(str(vals[0])) == 4:
        born = date(year=vals[0], month=vals[1], day=vals[2])
    else:
        born = date(year=vals[2], month=vals[1], day=vals[0])
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

users_df['age'] = users_df['bdate'].apply(get_age)

users_df.head()

,user_id,gender,bdate,age
3,1684318,male,1987-10-11,33.0
22,1290242,male,1982-09-24,38.0
55,632900,male,1989-03-16,31.0
103,2107573,male,1978-12-04,41.0
116,1781771,female,1948-01-10,72.0


### sample_sub

In [21]:
test_df = pd.read_csv(join(DATADIR, 'sample_submission.csv'), sep=';')
test_df.head()

,Id,Predicted
0,19843,NaN
1,471287,NaN
2,342522,NaN
3,457799,NaN
4,233778,NaN


## Обработка

### Соответствие phone_id-addr_id-user_id

In [22]:
ids_df = ship_df[['user_id', 'ship_address_id']].drop_duplicates().rename(columns={'ship_address_id':'address_id'})
ids_df.head()

,user_id,address_id
0,11019,171906
1,62278,278832
2,905126,468407
3,21412,61962
4,42110,378297


In [23]:
addr_df = addr_df.rename(columns={'id':'address_id'}).drop_duplicates()

In [24]:
ids_df = pd.merge(ids_df, addr_df, on='address_id', how='outer')
ids_df.head()

,user_id,address_id,phone_id
0,11019.0,171906,78177
1,62278.0,278832,96681
2,905126.0,468407,140317
3,21412.0,61962,16
4,42110.0,378297,60075


In [25]:
ids_df.to_csv('/Users/Anastasia/Desktop/python/Hackaton/DATA/ids.csv', index=False)

## Генерация признаков

In [27]:
ship_df['time_to_complete'] = (ship_df['order_completed_at'] - ship_df['order_created_at']).dt.days

In [26]:
def calculate_features(curr_ships, month, dt):
    features_df = pd.concat([
        curr_ships.groupby('phone_id').size().to_frame('n_orders'),
        curr_ships.groupby('phone_id')['order_id'].nunique().to_frame('n_total_orders'),
        curr_ships.groupby('phone_id')['time_to_complete'].agg(['min', 'mean', 'max', 'std']).add_prefix('time_to_complete_'),
        curr_ships.groupby('phone_id')['total_weight'].agg(['min', 'mean', 'max', 'std']).add_prefix('total_weight_'),
        curr_ships.groupby('phone_id')['total_cost'].agg(['min', 'mean', 'max', 'std']).add_prefix('total_cost_'),
        curr_ships.groupby('phone_id')['rate'].agg(['min', 'mean', 'max', 'std']).add_prefix('rate_'),
        pd.crosstab(index=curr_ships['phone_id'], columns=curr_ships['s.order_state']),
        (curr_ships.groupby('phone_id')['order_completed_at_month'].nunique() / (month-1)).to_frame('n_unique_months'),
        (dt - curr_ships.groupby('phone_id')['order_completed_at'].max()).dt.days.to_frame('last_order_days'),
        (dt - curr_ships.groupby('phone_id')['order_completed_at'].min()).dt.days.to_frame('first_order_days'),
        (curr_ships.groupby('phone_id')['order_completed_at'].max() - curr_ships.groupby('phone_id')['order_completed_at'].min()).dt.days.to_frame('client_days')
    ], axis=1).reset_index()

    
    return features_df

def get_completed_retro_features(month, n_days=None):
    
    dt = datetime(year=2020, month=month, day=1, hour=0, minute=0)
    
    if isinstance(n_days, int):
        curr_ships = ship_df[(ship_df['order_completed_at'] < dt) & (ship_df['order_completed_at'] > dt-timedelta(days=n_days))]
    else:
        curr_ships = ship_df[ship_df['order_completed_at'] < dt]
        
    curr_ships = curr_ships.merge(addr_df, left_on='ship_address_id', right_on='address_id')

    features_df = calculate_features(curr_ships, month=month, dt=dt)
    
    features_df['order_completed_at'] = month
    
    if isinstance(n_days, int):
        features_df = features_df.set_index(['phone_id', 'order_completed_at']).add_prefix(f'last_{n_days}_days_').reset_index()
    else:
        features_df = features_df.set_index(['phone_id', 'order_completed_at']).add_prefix('overall_').reset_index()
    
    return features_df

In [28]:
features_df_overall = pd.concat([get_completed_retro_features(month) for month in tqdm_notebook(range(2, 8))], axis=0)
features_df_last_month = pd.concat([get_completed_retro_features(month, n_days=30) for month in tqdm_notebook(range(2, 8))], axis=0)
#features_df_last_week = pd.concat([get_completed_retro_features(month, n_days=7) for month in tqdm_notebook(range(2, 8))], axis=0)

In [18]:
final_df = train_df.merge(features_df_overall, on=['phone_id', 'order_completed_at'], how='left')
final_df = final_df.merge(features_df_last_month, on=['phone_id', 'order_completed_at'], how='left')
#final_df = final_df.merge(features_df_last_week, on=['phone_id', 'order_completed_at'], how='left')

In [19]:
final_df['last_30_total_ratio'] = final_df['last_30_days_n_orders'] / final_df['overall_n_orders']
final_df['mean_days_per_order'] = (final_df['overall_client_days'] / final_df['overall_n_total_orders'])

### По возрасту

In [20]:
phone_age = ids_df.merge(users_df, on='user_id')[['phone_id', 'gender', 'age']].drop_duplicates()
phone_age['is_male'] = (phone_age['gender'] == 'male').astype(float)
phone_age = phone_age[['phone_id', 'age', 'is_male']].drop_duplicates('phone_id')
phone_age.head()

,phone_id,age,is_male
0,182858,17.0,1.0
1,180591,17.0,1.0
4,78475,25.0,1.0
6,49534,36.0,0.0
12,446979,36.0,0.0


In [21]:
final_df = final_df.merge(phone_age, on='phone_id', how='left')

### Статистика по заказам

Таргет 2, 3, 4 месяца назад:

In [22]:
two_years_ago = train_df.copy()
two_years_ago['order_completed_at'] += 2
two_years_ago.rename(columns={'target':'target_2m_ago'}, inplace=True)

three_years_ago = train_df.copy()
three_years_ago['order_completed_at'] += 3
three_years_ago.rename(columns={'target':'target_3m_ago'}, inplace=True)

four_years_ago = train_df.copy()
four_years_ago['order_completed_at'] += 4
four_years_ago.rename(columns={'target':'target_4m_ago'}, inplace=True)

In [23]:
final_df = final_df.merge(two_years_ago, on=['phone_id', 'order_completed_at'], how='left')
final_df = final_df.merge(three_years_ago, on=['phone_id', 'order_completed_at'], how='left')
final_df = final_df.merge(four_years_ago, on=['phone_id', 'order_completed_at'], how='left')

In [24]:
final_df = final_df[final_df['order_completed_at'] > 1]

In [25]:
final_df = final_df.drop(['last_30_days_cart', 'overall_cart', 'last_30_days_resumed', 'overall_resumed', 'last_30_days_last_order_days', 'last_30_days_n_unique_months', 'last_30_days_first_order_days', 'last_30_days_client_days'], axis=1)

## Модель

In [26]:
train = final_df[final_df['order_completed_at'] < 7]
test = final_df[final_df['order_completed_at'] == 7]

X_train, y_train = train.drop(['target', 'phone_id', 'order_completed_at'], axis=1), train['target']
X_test, y_test = test.drop(['target', 'phone_id', 'order_completed_at'], axis=1), test['target']

In [27]:
model = catboost.CatBoostClassifier()
model.fit(X_train, y_train)
pred = model.predict(X_test)

Learning rate set to 0.1117
0:	learn: 0.6814339	total: 309ms	remaining: 5m 9s
1:	learn: 0.6719574	total: 509ms	remaining: 4m 14s
2:	learn: 0.6644883	total: 729ms	remaining: 4m 2s
3:	learn: 0.6585165	total: 939ms	remaining: 3m 53s
4:	learn: 0.6536828	total: 1.14s	remaining: 3m 47s
5:	learn: 0.6498094	total: 1.33s	remaining: 3m 40s
6:	learn: 0.6467687	total: 1.53s	remaining: 3m 37s
7:	learn: 0.6441118	total: 1.73s	remaining: 3m 34s
8:	learn: 0.6420318	total: 1.93s	remaining: 3m 32s
9:	learn: 0.6402739	total: 2.14s	remaining: 3m 32s
10:	learn: 0.6388595	total: 2.33s	remaining: 3m 29s
11:	learn: 0.6375851	total: 2.51s	remaining: 3m 26s
12:	learn: 0.6366657	total: 2.72s	remaining: 3m 26s
13:	learn: 0.6357452	total: 2.9s	remaining: 3m 24s
14:	learn: 0.6350620	total: 3.1s	remaining: 3m 23s
15:	learn: 0.6345063	total: 3.29s	remaining: 3m 22s
16:	learn: 0.6340405	total: 3.47s	remaining: 3m 20s
17:	learn: 0.6335420	total: 3.68s	remaining: 3m 20s
18:	learn: 0.6330585	total: 3.87s	remaining: 3m 19

157:	learn: 0.6250776	total: 36s	remaining: 3m 11s
158:	learn: 0.6250571	total: 36.2s	remaining: 3m 11s
159:	learn: 0.6250446	total: 36.4s	remaining: 3m 11s
160:	learn: 0.6250267	total: 36.7s	remaining: 3m 11s
161:	learn: 0.6250097	total: 36.8s	remaining: 3m 10s
162:	learn: 0.6249886	total: 37s	remaining: 3m 10s
163:	learn: 0.6249737	total: 37.2s	remaining: 3m 9s
164:	learn: 0.6249405	total: 37.5s	remaining: 3m 9s
165:	learn: 0.6249208	total: 37.6s	remaining: 3m 9s
166:	learn: 0.6249079	total: 37.9s	remaining: 3m 8s
167:	learn: 0.6248918	total: 38.1s	remaining: 3m 8s
168:	learn: 0.6248764	total: 38.3s	remaining: 3m 8s
169:	learn: 0.6248440	total: 38.5s	remaining: 3m 8s
170:	learn: 0.6248265	total: 38.7s	remaining: 3m 7s
171:	learn: 0.6248127	total: 39s	remaining: 3m 7s
172:	learn: 0.6247937	total: 39.2s	remaining: 3m 7s
173:	learn: 0.6247738	total: 39.4s	remaining: 3m 7s
174:	learn: 0.6247575	total: 39.7s	remaining: 3m 7s
175:	learn: 0.6247420	total: 39.9s	remaining: 3m 6s
176:	learn: 

314:	learn: 0.6228861	total: 1m 9s	remaining: 2m 31s
315:	learn: 0.6228700	total: 1m 9s	remaining: 2m 31s
316:	learn: 0.6228555	total: 1m 10s	remaining: 2m 31s
317:	learn: 0.6228383	total: 1m 10s	remaining: 2m 30s
318:	learn: 0.6228288	total: 1m 10s	remaining: 2m 30s
319:	learn: 0.6228158	total: 1m 10s	remaining: 2m 30s
320:	learn: 0.6228019	total: 1m 11s	remaining: 2m 30s
321:	learn: 0.6227894	total: 1m 11s	remaining: 2m 30s
322:	learn: 0.6227821	total: 1m 11s	remaining: 2m 29s
323:	learn: 0.6227697	total: 1m 11s	remaining: 2m 29s
324:	learn: 0.6227613	total: 1m 11s	remaining: 2m 29s
325:	learn: 0.6227491	total: 1m 12s	remaining: 2m 29s
326:	learn: 0.6227336	total: 1m 12s	remaining: 2m 28s
327:	learn: 0.6227191	total: 1m 12s	remaining: 2m 28s
328:	learn: 0.6227103	total: 1m 12s	remaining: 2m 28s
329:	learn: 0.6227009	total: 1m 13s	remaining: 2m 28s
330:	learn: 0.6226909	total: 1m 13s	remaining: 2m 28s
331:	learn: 0.6226769	total: 1m 13s	remaining: 2m 27s
332:	learn: 0.6226674	total: 1

467:	learn: 0.6213145	total: 1m 42s	remaining: 1m 56s
468:	learn: 0.6213085	total: 1m 42s	remaining: 1m 55s
469:	learn: 0.6212955	total: 1m 42s	remaining: 1m 55s
470:	learn: 0.6212856	total: 1m 42s	remaining: 1m 55s
471:	learn: 0.6212718	total: 1m 42s	remaining: 1m 55s
472:	learn: 0.6212642	total: 1m 43s	remaining: 1m 54s
473:	learn: 0.6212543	total: 1m 43s	remaining: 1m 54s
474:	learn: 0.6212430	total: 1m 43s	remaining: 1m 54s
475:	learn: 0.6212324	total: 1m 43s	remaining: 1m 54s
476:	learn: 0.6212227	total: 1m 43s	remaining: 1m 53s
477:	learn: 0.6212115	total: 1m 44s	remaining: 1m 53s
478:	learn: 0.6211997	total: 1m 44s	remaining: 1m 53s
479:	learn: 0.6211847	total: 1m 44s	remaining: 1m 53s
480:	learn: 0.6211722	total: 1m 44s	remaining: 1m 52s
481:	learn: 0.6211662	total: 1m 44s	remaining: 1m 52s
482:	learn: 0.6211572	total: 1m 44s	remaining: 1m 52s
483:	learn: 0.6211515	total: 1m 45s	remaining: 1m 52s
484:	learn: 0.6211431	total: 1m 45s	remaining: 1m 51s
485:	learn: 0.6211353	total:

621:	learn: 0.6200475	total: 2m 12s	remaining: 1m 20s
622:	learn: 0.6200380	total: 2m 12s	remaining: 1m 20s
623:	learn: 0.6200289	total: 2m 12s	remaining: 1m 19s
624:	learn: 0.6200228	total: 2m 12s	remaining: 1m 19s
625:	learn: 0.6200109	total: 2m 12s	remaining: 1m 19s
626:	learn: 0.6200047	total: 2m 13s	remaining: 1m 19s
627:	learn: 0.6199921	total: 2m 13s	remaining: 1m 19s
628:	learn: 0.6199773	total: 2m 13s	remaining: 1m 18s
629:	learn: 0.6199692	total: 2m 14s	remaining: 1m 18s
630:	learn: 0.6199607	total: 2m 14s	remaining: 1m 18s
631:	learn: 0.6199518	total: 2m 14s	remaining: 1m 18s
632:	learn: 0.6199469	total: 2m 14s	remaining: 1m 18s
633:	learn: 0.6199359	total: 2m 14s	remaining: 1m 17s
634:	learn: 0.6199248	total: 2m 14s	remaining: 1m 17s
635:	learn: 0.6199128	total: 2m 15s	remaining: 1m 17s
636:	learn: 0.6199058	total: 2m 15s	remaining: 1m 17s
637:	learn: 0.6198970	total: 2m 15s	remaining: 1m 16s
638:	learn: 0.6198868	total: 2m 15s	remaining: 1m 16s
639:	learn: 0.6198800	total:

776:	learn: 0.6187065	total: 2m 50s	remaining: 49s
777:	learn: 0.6186998	total: 2m 50s	remaining: 48.7s
778:	learn: 0.6186892	total: 2m 50s	remaining: 48.5s
779:	learn: 0.6186782	total: 2m 51s	remaining: 48.3s
780:	learn: 0.6186676	total: 2m 51s	remaining: 48.1s
781:	learn: 0.6186648	total: 2m 51s	remaining: 47.9s
782:	learn: 0.6186603	total: 2m 52s	remaining: 47.7s
783:	learn: 0.6186528	total: 2m 52s	remaining: 47.5s
784:	learn: 0.6186435	total: 2m 52s	remaining: 47.3s
785:	learn: 0.6186373	total: 2m 52s	remaining: 47s
786:	learn: 0.6186278	total: 2m 53s	remaining: 46.9s
787:	learn: 0.6186174	total: 2m 53s	remaining: 46.6s
788:	learn: 0.6186085	total: 2m 53s	remaining: 46.4s
789:	learn: 0.6186041	total: 2m 53s	remaining: 46.2s
790:	learn: 0.6185986	total: 2m 54s	remaining: 46s
791:	learn: 0.6185986	total: 2m 54s	remaining: 45.8s
792:	learn: 0.6185910	total: 2m 54s	remaining: 45.5s
793:	learn: 0.6185791	total: 2m 54s	remaining: 45.4s
794:	learn: 0.6185726	total: 2m 55s	remaining: 45.1s

933:	learn: 0.6174493	total: 3m 30s	remaining: 14.9s
934:	learn: 0.6174388	total: 3m 30s	remaining: 14.7s
935:	learn: 0.6174337	total: 3m 31s	remaining: 14.4s
936:	learn: 0.6174206	total: 3m 31s	remaining: 14.2s
937:	learn: 0.6174131	total: 3m 31s	remaining: 14s
938:	learn: 0.6174066	total: 3m 31s	remaining: 13.8s
939:	learn: 0.6173951	total: 3m 32s	remaining: 13.5s
940:	learn: 0.6173885	total: 3m 32s	remaining: 13.3s
941:	learn: 0.6173816	total: 3m 32s	remaining: 13.1s
942:	learn: 0.6173749	total: 3m 32s	remaining: 12.9s
943:	learn: 0.6173679	total: 3m 33s	remaining: 12.7s
944:	learn: 0.6173638	total: 3m 33s	remaining: 12.4s
945:	learn: 0.6173546	total: 3m 33s	remaining: 12.2s
946:	learn: 0.6173473	total: 3m 33s	remaining: 12s
947:	learn: 0.6173395	total: 3m 34s	remaining: 11.7s
948:	learn: 0.6173357	total: 3m 34s	remaining: 11.5s
949:	learn: 0.6173356	total: 3m 34s	remaining: 11.3s
950:	learn: 0.6173308	total: 3m 34s	remaining: 11.1s
951:	learn: 0.6173239	total: 3m 35s	remaining: 10.

In [32]:
f1_score(y_test, pred)

0.6629112850004714

In [29]:
final_df = final_df.drop(['last_30_days_total_cost_std', 
                'last_30_days_total_cost_max',
                'last_30_days_total_cost_min',
                'last_30_days_rate_max',
                'overall_rate_max',
                'last_30_days_rate_min',
                'overall_rate_min'], axis=1)

In [30]:
X, y = final_df.drop(['target', 'phone_id', 'order_completed_at'], axis=1), final_df['target']


In [31]:
model = catboost.CatBoostClassifier()
model.fit(X, y)

Learning rate set to 0.121571
0:	learn: 0.6768900	total: 401ms	remaining: 6m 40s
1:	learn: 0.6644894	total: 677ms	remaining: 5m 37s
2:	learn: 0.6547901	total: 968ms	remaining: 5m 21s
3:	learn: 0.6473847	total: 1.36s	remaining: 5m 39s
4:	learn: 0.6416807	total: 1.75s	remaining: 5m 47s
5:	learn: 0.6371282	total: 2.07s	remaining: 5m 43s
6:	learn: 0.6334233	total: 2.4s	remaining: 5m 40s
7:	learn: 0.6306010	total: 2.72s	remaining: 5m 36s
8:	learn: 0.6281099	total: 3.07s	remaining: 5m 37s
9:	learn: 0.6261386	total: 3.54s	remaining: 5m 50s
10:	learn: 0.6246660	total: 3.86s	remaining: 5m 47s
11:	learn: 0.6233677	total: 4.23s	remaining: 5m 47s
12:	learn: 0.6223557	total: 4.59s	remaining: 5m 48s
13:	learn: 0.6213625	total: 5.02s	remaining: 5m 53s
14:	learn: 0.6206856	total: 5.42s	remaining: 5m 55s
15:	learn: 0.6198907	total: 5.81s	remaining: 5m 57s
16:	learn: 0.6192650	total: 6.09s	remaining: 5m 52s
17:	learn: 0.6188417	total: 6.41s	remaining: 5m 49s
18:	learn: 0.6184339	total: 6.83s	remaining: 

158:	learn: 0.6093882	total: 55.4s	remaining: 4m 52s
159:	learn: 0.6093659	total: 55.7s	remaining: 4m 52s
160:	learn: 0.6093474	total: 56.1s	remaining: 4m 52s
161:	learn: 0.6093326	total: 56.4s	remaining: 4m 51s
162:	learn: 0.6093161	total: 56.7s	remaining: 4m 51s
163:	learn: 0.6092973	total: 57.1s	remaining: 4m 51s
164:	learn: 0.6092892	total: 57.5s	remaining: 4m 50s
165:	learn: 0.6092652	total: 57.8s	remaining: 4m 50s
166:	learn: 0.6092533	total: 58.1s	remaining: 4m 50s
167:	learn: 0.6092330	total: 58.4s	remaining: 4m 49s
168:	learn: 0.6092199	total: 58.7s	remaining: 4m 48s
169:	learn: 0.6091948	total: 59.2s	remaining: 4m 48s
170:	learn: 0.6091739	total: 59.4s	remaining: 4m 48s
171:	learn: 0.6091589	total: 59.8s	remaining: 4m 47s
172:	learn: 0.6091461	total: 1m	remaining: 4m 47s
173:	learn: 0.6091278	total: 1m	remaining: 4m 47s
174:	learn: 0.6091128	total: 1m	remaining: 4m 47s
175:	learn: 0.6090892	total: 1m 1s	remaining: 4m 46s
176:	learn: 0.6090733	total: 1m 1s	remaining: 4m 46s
17

312:	learn: 0.6074028	total: 1m 47s	remaining: 3m 57s
313:	learn: 0.6073928	total: 1m 48s	remaining: 3m 56s
314:	learn: 0.6073784	total: 1m 48s	remaining: 3m 56s
315:	learn: 0.6073671	total: 1m 49s	remaining: 3m 55s
316:	learn: 0.6073556	total: 1m 49s	remaining: 3m 55s
317:	learn: 0.6073345	total: 1m 49s	remaining: 3m 55s
318:	learn: 0.6073249	total: 1m 50s	remaining: 3m 55s
319:	learn: 0.6073135	total: 1m 50s	remaining: 3m 54s
320:	learn: 0.6073035	total: 1m 50s	remaining: 3m 54s
321:	learn: 0.6072948	total: 1m 51s	remaining: 3m 54s
322:	learn: 0.6072837	total: 1m 51s	remaining: 3m 53s
323:	learn: 0.6072764	total: 1m 51s	remaining: 3m 53s
324:	learn: 0.6072703	total: 1m 52s	remaining: 3m 52s
325:	learn: 0.6072644	total: 1m 52s	remaining: 3m 52s
326:	learn: 0.6072529	total: 1m 52s	remaining: 3m 52s
327:	learn: 0.6072449	total: 1m 53s	remaining: 3m 51s
328:	learn: 0.6072372	total: 1m 53s	remaining: 3m 51s
329:	learn: 0.6072201	total: 1m 53s	remaining: 3m 51s
330:	learn: 0.6072109	total:

465:	learn: 0.6059252	total: 2m 40s	remaining: 3m 4s
466:	learn: 0.6059158	total: 2m 40s	remaining: 3m 3s
467:	learn: 0.6059060	total: 2m 41s	remaining: 3m 3s
468:	learn: 0.6058928	total: 2m 41s	remaining: 3m 2s
469:	learn: 0.6058866	total: 2m 41s	remaining: 3m 2s
470:	learn: 0.6058780	total: 2m 42s	remaining: 3m 2s
471:	learn: 0.6058650	total: 2m 42s	remaining: 3m 1s
472:	learn: 0.6058566	total: 2m 42s	remaining: 3m 1s
473:	learn: 0.6058471	total: 2m 43s	remaining: 3m 1s
474:	learn: 0.6058379	total: 2m 43s	remaining: 3m
475:	learn: 0.6058256	total: 2m 43s	remaining: 3m
476:	learn: 0.6058166	total: 2m 44s	remaining: 3m
477:	learn: 0.6058154	total: 2m 44s	remaining: 2m 59s
478:	learn: 0.6058076	total: 2m 44s	remaining: 2m 59s
479:	learn: 0.6057977	total: 2m 45s	remaining: 2m 58s
480:	learn: 0.6057845	total: 2m 45s	remaining: 2m 58s
481:	learn: 0.6057776	total: 2m 45s	remaining: 2m 58s
482:	learn: 0.6057719	total: 2m 46s	remaining: 2m 57s
483:	learn: 0.6057619	total: 2m 46s	remaining: 2m

618:	learn: 0.6046410	total: 3m 31s	remaining: 2m 10s
619:	learn: 0.6046378	total: 3m 32s	remaining: 2m 10s
620:	learn: 0.6046329	total: 3m 32s	remaining: 2m 9s
621:	learn: 0.6046253	total: 3m 32s	remaining: 2m 9s
622:	learn: 0.6046171	total: 3m 33s	remaining: 2m 8s
623:	learn: 0.6046163	total: 3m 33s	remaining: 2m 8s
624:	learn: 0.6046082	total: 3m 33s	remaining: 2m 8s
625:	learn: 0.6045971	total: 3m 34s	remaining: 2m 7s
626:	learn: 0.6045865	total: 3m 34s	remaining: 2m 7s
627:	learn: 0.6045756	total: 3m 34s	remaining: 2m 7s
628:	learn: 0.6045624	total: 3m 35s	remaining: 2m 6s
629:	learn: 0.6045601	total: 3m 35s	remaining: 2m 6s


KeyboardInterrupt: 

In [49]:
model.save_model('model.pickle')

## Накатим на тест

In [18]:
test_df = test_df[['Id']]
test_df

,Id
0,19843
1,471287
2,342522
3,457799
4,233778
...,...
214604,16841
214605,69348
214606,594974
214607,320024


In [19]:
test_features_df_overall = get_completed_retro_features(9)
test_features_df_last_month = get_completed_retro_features(9, n_days=30)

In [20]:
test_df = test_df.merge(test_features_df_overall, left_on='Id', right_on='phone_id', how='left').merge(test_features_df_last_month, left_on='Id', right_on='phone_id', how='left').drop(['phone_id_x', 'order_completed_at_x', 'phone_id_y', 'order_completed_at_y'], axis=1)

In [21]:
test_df['last_30_total_ratio'] = test_df['last_30_days_n_orders'] / test_df['overall_n_orders']
test_df['mean_days_per_order'] = (test_df['overall_client_days'] / test_df['overall_n_total_orders'])

In [24]:
test_df = test_df.merge(phone_age, left_on='Id', right_on='phone_id', how='left')

In [27]:
model = catboost.CatBoostClassifier()
model = model.load_model('./model.pickle')

In [28]:
set(model.feature_names_) - set(test_df)

set()

In [31]:
test_df['pred'] = model.predict(test_df[model.feature_names_])

In [44]:
f = test_df[['Id', 'pred']].rename(columns={'pred':'Predicted'})
f.head()

,Id,Predicted
0,19843,0.0
1,471287,1.0
2,342522,0.0
3,457799,1.0
4,233778,0.0


In [45]:
sample = pd.read_csv(join(DATADIR, 'sample_submission_from_kaggle_change_sep.csv'))
sample.head()

,Id,Predicted
0,19843,1
1,471287,1
2,342522,1
3,457799,1
4,233778,1


In [46]:
f.to_csv('./sub1.csv', index=False)